# <center> Purchasing your next house! </center>

### Libraries, credentials, and configuration

In [194]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
import pprint
import folium
from folium.features import DivIcon

# Four Square credentials
client_id = 'PEQPJEZXPG4HRTFTMTZHL3P3QRZ1SV55525PX2YO22F3Q0JB'
client_secret = 'JEDPFGTRY4SISHI1S2W3RTQGIG3UUROA5OG4I3NW4UXBBLB1'
version = '20180605'

# OpenCage Geocoding API key
my_api_key = "a91cf6a6d9ec4e6fa91deb0bc4fbf168"

### Shortlisting the cities based on population

There are some good cities around Austin, but it will become a heavy task to compare and contrast each cities as there are more than 30. So, this project will be limited to top three cities with a high population in the list.

In [2]:
# Scraping the near-by cities list from wikipedia
austin_nearby = "https://en.wikipedia.org/wiki/Greater_Austin"
response = requests.get(austin_nearby)

# Parsing the results with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find_all("table", class_="wikitable")
population_table = table[1]
pop_def = population_table.find_all('td')

city_list = []
population_list = []

for i in range(0, 27, 3):
    city_list.append(pop_def[i].text.strip())
    population_list.append(pop_def[i+1].contents[0])

city_df = pd.DataFrame(columns=["CityName", "Population"])
city_df["CityName"] = city_list 
city_df["Population"] = population_list
city_df

,CityName,Population
0,Austin,"950,715"
1,Round Rock,"123,678"
2,Cedar Park,"75,704"
3,Georgetown,"70,685"
4,San Marcos,"63,071"
5,Pflugerville,"59,245"
6,Leander,"42,761"
7,Kyle,"39,060"
8,Hutto,"23,832"


As we see, the top three cities are: <strong>Austin, Round Rock, and Cedar Park</strong>. So, let's start exploring them


In [3]:
# Getting the latitude and longitude coordinates of these cities using OpenCage Geocoding APIs
city_list = ['Austin', 'Round Rock', 'Cedar Park']
latitude_list = []
longitude_list = []
top_city_df = pd.DataFrame(columns=["CityName", "Population", "Latitude", "Longitude"])

for city in city_list:
    query = city + ", TX, United States of America"
    uri = "https://api.opencagedata.com/geocode/v1/json?q={}&key={}".format(query, my_api_key)
    response = requests.get(uri).json()
    latitude_list.append(response['results'][0]['geometry']['lat'])
    longitude_list.append(response['results'][0]['geometry']['lng'])
    
top_city_df["CityName"] = city_list
top_city_df["Latitude"] = latitude_list
top_city_df["Population"] = city_df["Population"]
top_city_df["Longitude"] = longitude_list
    
top_city_df

,CityName,Population,Latitude,Longitude
0,Austin,"950,715",30.271129,-97.743700
1,Round Rock,"123,678",30.508235,-97.678893
2,Cedar Park,"75,704",30.521712,-97.827833


### Exploring the top 3 cities with respect to various attributes

For all three cities, we are going to explore different aspects using FourSquare and other APIs to get further insights into a particular city. These different aspects could be a good starting point to consider when purchasing a house:
<ul>
    <li>Number of schools with High-rating</li>
    <li>Average prices of the house</li>
    <li>Property taxes</li>
    <li>Average HOA</li>
    <li>Health  [Hospital, Pharmacies, Clinics, Dentist]</li>
    <li>Food  [Grocery shops, restaurants, coffee shops]</li>
    <li>Entertainment  [Movie Theaters, Bars]</li>
</ul>



In [189]:
# Defining a general function that will be used to consume FourSquare APIs to get details about various categories
def getCategories(latitude, longitude, query):
    num_of_venues = 100
    radius = 500
    categories_uri = "https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}".format(client_id, client_secret, version, latitude, longitude, radius, num_of_venues, query)
    return categories_uri

<center><strong>Austin, TX</strong></center>

#### i. HEALTH - that includes hospitals, pharmacies, clinic, dentists etc.

In [191]:
# Writing code to consume the above-function, and store the health responses to a dataframe
health_list = ["hospital", "pharmacy", "medical", "clinic"]
austin_health_dataframe = pd.DataFrame(columns=["Health Venue", "Latitude", "Longitude"])
venue = []
latitude = []
longitude = []
for h in health_list:
        response = requests.get(getCategories(top_city_df.loc[0, 'Latitude'], top_city_df.loc[0, 'Longitude'], h)).json()
        for i in range(len(response['response']['venues'])):
            venue.append(response['response']['venues'][i]['name'])
            latitude.append(response['response']['venues'][i]['location']['lat'])
            longitude.append(response['response']['venues'][i]['location']['lng'])

austin_health_dataframe["Health Venue"] = venue
austin_health_dataframe["Latitude"] = latitude
austin_health_dataframe["Longitude"] = longitude

austin_health_dataframe      

,Health Venue,Latitude,Longitude
0,Austin Shoe Hospital,30.269806,-97.742216
1,Texas Hospital Association,30.274438,-97.743323
2,Texas Hospital Association,30.274274,-97.743381
3,CVS pharmacy,30.267245,-97.743364
4,Brown Wharton & Brothers Medical Malpractice L...,30.268149,-97.742321
5,Medical WasteX Disposal,30.271077,-97.743709
6,Art Medical Alliance,30.270931,-97.741763
7,Texas Medical Board,30.270188,-97.746904
8,Dell Children's Medical Center,30.267548,-97.739952
9,Texas State Board of Veterinary Medical Examiners,30.266288,-97.745539


In [260]:
# visualizing health venues of Austin 

# creating a map centralized to Austin
austin_map = folium.Map(location=[30.271129, -97.743700], min_zoom=15)

# marking Austin health facilities
for lat, long, venue in zip(austin_health_dataframe["Latitude"], austin_health_dataframe["Longitude"], austin_health_dataframe["Health Venue"]):
    folium.CircleMarker(location=[lat, long],
    radius=8,
    popup=venue,
    color='#FF0000',
    fill=True,
    fill_color='#FF0000',
    fill_opacity=0.5).add_to(austin_map)
    folium.map.Marker([30.254, -97.738],
    icon=DivIcon(
        icon_size=(500,500),
        icon_anchor=(0,0),
        html='<p style="font-size:24px;"><strong>Austin Health Venues</strong></p>',)).add_to(austin_map)


austin_map
